In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
import numpy as np
import seaborn as sns
import os
import fastparquet
import warnings
import geopy
from geopy.point import Point
import time
from pandas.core.common import SettingWithCopyWarning
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta, date
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
# Load in price data
price_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/prices')

In [3]:
price_data.id = price_data.id.astype('str')

In [77]:
miami_example = price_data[price_data['id'] == '52364652.0']

In [79]:
start_time = time.time()
miami_example['pull_time'] = pd.to_datetime(miami_example['pull_time'])
print('done with datatype change')
middle_time = time.time()
miami_example['pull_time'] = miami_example['pull_time'].dt.date
print('done with change to date')
end_time = time.time()

print('Start Time '+ str(start_time))
print('Middle Time '+ str(middle_time-start_time))
print('End Time '+ str(end_time-start_time))


done with datatype change
done with change to date
Start Time 1656192768.4057891
Middle Time 0.056856632232666016
End Time 0.08577251434326172


In [6]:
miami_example_by_pull_time = miami_example.groupby(['id','pull_time'])['cleaning_fee','service_fee','total_price'].mean().reset_index()
miami_example_by_pull_time_count = miami_example.groupby(['pull_time','check_in'])['cleaning_fee','service_fee','total_price'].count().reset_index()

In [8]:
# Create traces
miami_example_grouped_fig = go.Figure()
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pull_time'], y=miami_example_by_pull_time['cleaning_fee'],
                    mode='lines',
                    name='Cleaning Fee'))
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pull_time'], y=miami_example_by_pull_time['service_fee'],
                    mode='lines',
                    name='Service Fee'))
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pull_time'], y=miami_example_by_pull_time['total_price'],
                    mode='lines', 
                    name='Total Price'))
miami_example_grouped_fig.update_layout(title_text="Pricing Data by Date", xaxis_title='Date', yaxis_title='Price ($)')
filename = "newsletter_features/price_data_line_fig_miami_example.png"
miami_example_grouped_fig.write_image(filename, engine='kaleido', width=2000, height=500)
miami_example_grouped_fig.show()

In [11]:
import plotly.express as px
fig = px.histogram(miami_example_by_pull_time_count, x="pull_time",nbins=50)
fig.show()

In [62]:
# Reference https://plotly.com/python/histograms/ under "Accessing the counts (y-axis) values"

df = px.data.tips()
# create the bins
counts, bins = np.histogram(.total_bill, bins=range(0, 60, 5))
bins = 0.5 * (bins[:-1] + bins[1:])

fig = px.bar(x=bins, y=counts, labels={'x':'total_bill', 'y':'count'})
fig.show()

,id,pull_time,cleaning_fee,service_fee,total_price
0,52364652.0,2022-01-05,89,89,89
1,52364652.0,2022-02-05,89,89,89
2,52364652.0,2022-03-05,89,89,89
3,52364652.0,2022-04-05,89,89,89
4,52364652.0,2022-04-18,9,9,9
5,52364652.0,2022-04-19,98,98,98
6,52364652.0,2022-04-20,89,89,89
7,52364652.0,2022-04-21,89,89,89
8,52364652.0,2022-04-22,89,89,89
9,52364652.0,2022-04-23,89,89,89


In [12]:
# aggregate price data by mean and std grouped by id

agg_price_data = price_data.groupby(['id']).agg({'cleaning_fee' :['mean','std'], 'service_fee' :['mean','std'], 'total_price' :['mean','std']}).reset_index()

In [60]:
### Miami example for abstracting price increase
#miami_example[miami_example['check_in'] == '2022-04-19']
#miami_example.pull_time.unique()
#todays_date = datetime.date.today()
days_to_subtract = 30
last_day_from = pd.to_datetime('05/31/2022')
date_30_days_ago = last_day_from - timedelta(days=days_to_subtract)
pull_time_current = pd.to_datetime('04/20/2022')
pull_time_old = pd.to_datetime('05/05/2022')

miami_example_last30 = miami_example[(miami_example['check_in'] > str(date_30_days_ago)) & (miami_example['check_in'] <= str(last_day_from))]
miami_example_previous_pull_avg = miami_example_last30[miami_example_last30['pull_time'] == pull_time_old].mean()
miami_example_current_pull_avg = miami_example_last30[miami_example_last30['pull_time'] == pull_time_current].mean()
price_change_total = miami_example_current_pull_avg-miami_example_previous_pull_avg
price_change_pct = price_change_total/miami_example_previous_pull_avg

In [80]:
### Abstracting price increases on all data
days_to_subtract = 30
last_day_from = pd.to_datetime('05/31/2022')
date_30_days_ago = last_day_from - timedelta(days=days_to_subtract)
pull_time_current = pd.to_datetime('04/20/2022')
pull_time_old = pd.to_datetime('05/05/2022')

start_time = time.time()
price_data['pull_time'] = pd.to_datetime(price_data['pull_time'])
print('done with datatype change')
middle_time = time.time()
price_data['pull_time'] = price_data['pull_time'].dt.date
print('done with change to date')
end_time = time.time()
print('Start Time '+ str(start_time))
print('Middle Time '+ str(middle_time-start_time))
print('End Time '+ str(end_time-start_time))



done with datatype change
done with change to date
Start Time 1656192909.7642343
Middle Time 2615.258655309677
End Time 2620.6761894226074


In [85]:
import fastparquet
#price_data.to_feather('price_data_feather.feather')
price_data.to_parquet('price_data_parquet.parquet')

ValueError: Can't infer object conversion type: 0    2022-04-18
1    2022-04-18
2    2022-04-18
3    2022-04-18
4    2022-04-18
5    2022-04-18
6    2022-04-18
7    2022-04-18
8    2022-04-18
9    2022-04-18
Name: pull_time, dtype: object

In [86]:
price_data_last30 = price_data[(price_data['check_in'] > str(date_30_days_ago)) & (price_data['check_in'] <= str(last_day_from))]
print(price_data_last30.shape)
#price_data_last30_avgs = price_data_last30.groupby(['id','pull_time']).mean().reset_index()
price_data_previous_pull = price_data_last30[price_data_last30['pull_time'] == pull_time_old]
print(price_data_previous_pull.shape)
price_data_current_pull = price_data_last30[price_data_last30['pull_time'] == pull_time_current]
print(price_data_current_pull.shape)
price_data_previous_pull_avgs = price_data_previous_pull.groupby('id').mean().reset_index()
print(price_data_previous_pull_avgs.shape)
price_data_current_pull_avgs = price_data_current_pull.groupby('id').mean().reset_index()
print(price_data_current_pull_avgs.shape)
price_trend = price_data_previous_pull_avgs.merge(price_data_current_pull_avgs, how='inner', on='id')
print(price_trend.shape)

(2966455, 11)
(149241, 11)
(50097, 11)
(5530, 5)
(1671, 5)
(1633, 9)


In [96]:
# Do a delta from month of June price vs the month of May price for all properties to see increase vs decrease Month over Month
# Then, merge each id with its zipcode, state, and city (if city is there) from the flat csv files used in the eda_summary (listings_w_zips.csv)
# Then, groupby on zipcode with mean on delta from June-May to get average price increase vs decrease in that state/zip/city
"""
price_data_last30_avgs = price_data_last30.groupby(['id','pull_time']).mean().reset_index()
price_data_last30_avgs.to_csv('price_data_last30_avgs.csv')
data_w_listing_loc = pd.read_csv('post_mort_and_roi_calcs_june19.csv')
data_w_listing_loc = data_w_listing_loc[['id','zipcode','City','Metro','CountyName','State']]
price_data_last30_avgs.id = price_data_last30_avgs.id.astype('float64')
price_data_last30_avgs_w_loc = price_data_last30_avgs.merge(data_w_listing_loc, how='inner', on='id')
florida_price_trends = price_data_last30_avgs_w_loc[price_data_last30_avgs_w_loc['State'] == 'FL']
florida_price_trends = florida_price_trends.groupby('pull_time').mean(['cleaning_fee','service_fee','total_price']).reset_index()

#price_data_date = price_data.groupby(['check_in'])['cleaning_fee','service_fee','total_price'].apply(lambda row: np.sum(row)/len(row)).reset_index()

# Create traces
florida_price_trends_fig = go.Figure()
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['cleaning_fee'],
                    mode='lines',
                    name='Cleaning Fee'))
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['service_fee'],
                    mode='lines',
                    name='Service Fee'))
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['total_price'],
                    mode='lines', 
                    name='Total Price'))
florida_price_trends_fig.update_layout(title_text="Pricing Data by Pull Time", xaxis_title='Date', yaxis_title='Price ($)')
filename = "newsletter_features/florida_price_trends_fig_june19.png"
florida_price_trends_fig.write_image(filename, engine='kaleido', width=2000, height=500)
florida_price_trends_fig.show()
"""

KeyError: 'pull_time'

In [134]:
price_trend['cleaning_fee_delta'] = price_trend['cleaning_fee_y'] - price_trend['cleaning_fee_x']
price_trend['service_fee_delta'] = price_trend['service_fee_y'] - price_trend['service_fee_x']
price_trend['total_price_delta'] = price_trend['total_price_y'] - price_trend['total_price_x']
price_trend['display_price_delta'] = price_trend['total_price_delta'] - price_trend['service_fee_delta'] - price_trend['cleaning_fee_delta']
price_trend['cleaning_fee_delta_pct'] = price_trend['cleaning_fee_delta']/price_trend['cleaning_fee_x']*100
price_trend['service_fee_delta_pct'] = price_trend['service_fee_delta']/price_trend['service_fee_x']*100
price_trend['total_price_delta_pct'] = price_trend['total_price_delta']/price_trend['total_price_x']*100
price_trend['display_price_delta_pct'] = price_trend['display_price_delta']/price_trend['total_price_x']*100
data_w_listing_loc = pd.read_csv('post_mort_and_roi_calcs_june19.csv')
data_w_listing_loc = data_w_listing_loc[['id','zipcode','City','Metro','CountyName','State']]
price_trend['id'] = price_trend['id'].astype('float64')
price_trend_wloc = price_trend.merge(data_w_listing_loc, how='inner', on='id')

In [138]:
florida_price_trends = price_trend_wloc[price_trend_wloc['State'] == 'FL']
florida_price_trends_zip = florida_price_trends.groupby('zipcode').mean(['cleaning_fee_delta_pct','service_fee_delta_pct','total_price_delta_pct']).reset_index()
florida_price_trends_city = florida_price_trends.groupby('City').mean(['cleaning_fee_delta_pct','service_fee_delta_pct','total_price_delta_pct']).reset_index()
"""
florida_price_trends_fig = go.Figure()
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['cleaning_fee'],
                    mode='lines',
                    name='Cleaning Fee'))
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['service_fee'],
                    mode='lines',
                    name='Service Fee'))
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['total_price'],
                    mode='lines', 
                    name='Total Price'))
florida_price_trends_fig.update_layout(title_text="Pricing Data by Pull Time", xaxis_title='Date', yaxis_title='Price ($)')
filename = "newsletter_features/florida_price_trends_fig_june19.png"
florida_price_trends_fig.write_image(filename, engine='kaleido', width=2000, height=500)
florida_price_trends_fig.show()
"""
import plotly.graph_objects as go

zips = florida_price_trends_zip['zipcode']
vals = [florida_price_trends_zip['display_price_delta_pct'], florida_price_trends_zip['cleaning_fee_delta_pct'], florida_price_trends_zip['service_fee_delta_pct'], florida_price_trends_zip['total_price_delta_pct']]
font_color = ['rgb(40,40,40)'] +  [['rgb(255,0,0)' if v < 0 else 'rgb(0,125,0)' for v in vals[k]] for k in range(4)]

table_trace = go.Table(
                 columnwidth= [50]+[50]+[50]+[50]+[50],
                 columnorder=[0, 1, 2, 3, 4],
                 header = dict(height = 40,
                               values = [['<b>Zip Code</b>'], ['<b>Display Price</b>'], ['<b>Cleaning Fee</b>'], ['<b>Service Fee</b>'],['<b>Total Price</b>']],
                               line = dict(color='rgb(50,50,50)'),
                               align = ['left']*5,
                               font = dict(color=['rgb(45,45,45)']*4, size=14),
                             
                              ),
                 cells = dict(values = [zips, vals[0], vals[1], vals[2], vals[3]],
                              line = dict(color='#506784'),
                              align = ['left']*5,
                              
                              font = dict(family="Arial", size=14, color=font_color),
                              format = [None, ",.2f"],  #add % sign here
                              height = 30,
                              fill = dict(color='rgb(245,245,245)'))
                             )
                 

layout = go.Layout(width=850, height=650, autosize=False, 
              title_text='Recent Pricing Trends in Miami (%) (M/M)',
                   title_x=0.5, showlegend=False)
florda_price_trend_fig = go.Figure(data=[table_trace], layout=layout)
filename = "newsletter_features/florida_price_trends_table_june19.png"
florda_price_trend_fig.write_image(filename, engine='kaleido', width=875, height=700)
florda_price_trend_fig.show()

# Look to do one price value and look and price change by day of the week

In [95]:
# make a new notebook called occupancy_eda.ipynb and copy all this code except switch out occupancy and change up logic and table (maybe do like month over month occupancy or 3 month outlook on occupancy)
# (could be like occupancy rate and if its green it went up and if its red it went down)

In [97]:
price_trend

,id,index_x,cleaning_fee_x,service_fee_x,total_price_x,index_y,cleaning_fee_y,service_fee_y,total_price_y
0,10121456.0,0.0,488.0,481.346154,2921.923077,0.0,488.0,538.866667,3330.266667
1,10123207.0,0.0,NaN,64.740741,457.370370,0.0,NaN,105.133333,744.533333
2,10162954.0,0.0,250.0,164.259259,911.592593,0.0,250.0,174.900000,987.333333
3,10192685.0,0.0,215.0,266.888889,1675.925926,0.0,215.0,319.733333,2050.000000
4,10250283.0,0.0,170.0,64.000000,280.000000,0.0,170.0,64.000000,280.000000
...,...,...,...,...,...,...,...,...,...
1628,9839886.0,0.0,20.0,12.074074,65.555556,0.0,20.0,12.066667,65.500000
1629,9928289.0,0.0,NaN,124.555556,882.259259,0.0,NaN,134.300000,951.633333
1630,9940136.0,0.0,100.0,52.000000,266.000000,0.0,100.0,52.000000,266.000000
1631,994967.0,0.0,150.0,39.000000,125.000000,0.0,150.0,39.000000,125.000000
